In [96]:
# ваш код здесь


import pandas as pd
import numpy as np

log = pd.read_csv("log.csv",header=None)
log.columns = ['user_id', 'time', 'bet', 'win']
log = log[log['user_id']!='#error']
log['user_id'] = log['user_id'].apply(lambda x:x.replace('Запись пользователя № -','').strip())
log['time'] = log['time'].apply(lambda x:x.replace('[',''))
log['time'] = pd.to_datetime(log['time'])
log['bet'].fillna(0,inplace=True)
log['win'].fillna(0,inplace=True)

In [97]:
log.head()

,user_id,time,bet,win
0,user_919,2019-01-01 14:06:51,0.0,0.0
1,user_973,2019-01-01 14:51:16,0.0,0.0
2,user_903,2019-01-01 16:31:16,0.0,0.0
3,user_954,2019-01-01 17:17:51,0.0,0.0
4,user_954,2019-01-01 21:31:18,0.0,0.0


In [98]:
users = pd.read_csv('users.csv',encoding='koi8_r',sep='\t')
users.columns = ['user_id','email','city']
users['user_id'] = users['user_id'].apply(lambda x:x.lower().strip())

In [99]:
users_log = pd.merge(log,users,how='inner',on='user_id')

In [100]:
users_log.head()

,user_id,time,bet,win,email,city
0,user_919,2019-01-01 14:06:51,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
1,user_919,2019-01-30 10:06:00,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
2,user_919,2019-02-05 14:33:44,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
3,user_919,2019-02-14 11:38:05,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
4,user_919,2019-03-02 04:23:36,300.0,0.0,Chikkaverle@icloud.com,Хабаровск


In [101]:
first_bet_time   = users_log[users_log['bet']>0].groupby('user_id',as_index=False)['time'].min()
first_bet_time.rename({'time':'first_bet_time'},axis=1,inplace=True)
first_bet_time.head()
first_visit_time = users_log[users_log['bet']==0].groupby('user_id',as_index=False)['time'].min()
first_visit_time.rename({'time':'first_visit_time'},axis=1,inplace=True)
first_bet = pd.merge(first_visit_time,first_bet_time,'inner','user_id')
first_bet['before_first_bet'] = first_bet['first_bet_time'] - first_bet['first_visit_time']
first_bet['days_before_first_bet'] = first_bet['before_first_bet'].dt.days
first_bet.head()

,user_id,first_visit_time,first_bet_time,before_first_bet,days_before_first_bet
0,user_1000,2019-01-21 19:31:34,2019-01-28 20:57:37,7 days 01:26:03,7
1,user_900,2019-01-01 04:37:25,2019-02-08 10:36:47,38 days 05:59:22,38
2,user_901,2019-01-07 08:45:59,2019-03-14 13:51:20,66 days 05:05:21,66
3,user_902,2019-01-28 03:12:06,2019-03-09 16:52:11,40 days 13:40:05,40
4,user_903,2019-01-01 16:31:16,2019-02-08 11:06:00,37 days 18:34:44,37


In [102]:
first_bet['days_before_first_bet'].mean()

45.79

In [103]:
mean_bet = users_log[users_log['bet']>0].groupby('city',as_index=False)['bet'].mean()


In [104]:
mean_bet.head()



,city,bet
0,Арзангелтск,1415.911111
1,Воронеж,2119.745455
2,Екатеринбург,486.956522
3,Ижевск,2058.437500
4,Казань,1572.066667


In [105]:
max_city_bet = mean_bet['bet'].max()
print(max_city_bet)

62240.846153846156


In [106]:
min_city_bet = mean_bet['bet'].min()
print(min_city_bet)

486.95652173913044


In [107]:
max_city_bet/min_city_bet

127.81602335164835